<a href="https://colab.research.google.com/github/darrengao628/gptchem/blob/main/Fine_tune_ESM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers evaluate datasets requests pandas sklearn huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 K

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
def data_prep(seq):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq)
    df.dropna()
    cyto=df["Subcellular location [CC]"].str.contains("Cytosol") | df["Subcellular location [CC]"].str.contains("Cytoplasm")
    mem=df["Subcellular location [CC]"].str.contains("Membrane") | df["Subcellular location [CC]"].str.contains("Cell membrane")
    cyto_df=df[cyto& ~mem]
    cyto_seq=cyto_df["Sequence"].tolist()
    cyto_labels=[0 for protein in cyto_seq]
    mem_df=df[mem& ~cyto]
    mem_seq=mem_df["Sequence"].tolist()
    mem_labels=[1 for protein in mem_seq]
    seq = cyto_seq + mem_seq
    labels=cyto_labels + mem_labels
    return seq,labels

In [4]:
seq,labels=data_prep("/content/500_sequences.csv")
len(labels),len(seq)

(279, 279)

In [5]:
def data_loader_prep (seq, labels,model_checkpoint):
    from sklearn.model_selection import train_test_split
    from transformers import AutoTokenizer
    from datasets import Dataset
    train_seq, test_seq, train_labels, test_labels = train_test_split(seq, labels, test_size =0.25,shuffle=True)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    train_tokenized=tokenizer(train_seq)
    test_tokenized=tokenizer(test_seq)
    train_ds=Dataset.from_dict(train_tokenized)
    test_ds=Dataset.from_dict(test_tokenized)
    train_ds = train_ds.add_column("labels", train_labels)
    test_ds = test_ds.add_column("labels", test_labels)
    return train_ds, train_labels, test_ds,test_labels
    

In [6]:
model_checkpoint="facebook/esm2_t12_35M_UR50D"
train_ds, train_labels, test_ds,test_labels = data_loader_prep (seq, labels, model_checkpoint)
train_ds,test_ds 

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 209
 }), Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 70
 }))

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
num_labels = max(train_labels + test_labels) + 1  # Add 1 since 0 can be a label
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at facebook/esm2_t12_35M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight'

In [10]:
model_name=model_checkpoint.split('/')[-1]

In [11]:
batch_size = 8

args = TrainingArguments(
    f"{model_name}_Finetuned_localization_March_14_colab",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

from evaluate import load
import numpy as np

metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [15]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 209
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 81
  Number of trainable parameters = 33993843


Epoch,Training Loss,Validation Loss
